In [30]:
# Import Libraries and Dependencies
import pandas as pd
#Specifying file path
resource_dir = 'Resources/'

### 1. Combine and Clean the Data
#### Import CSVs

In [31]:
# Read the CSV files into DataFrames.
df_2020 = pd.read_csv(resource_dir + 'athletic_sales_2020.csv')
df_2021 = pd.read_csv(resource_dir + 'athletic_sales_2021.csv')

ERROR:tornado.general:SEND Error: Host unreachable


In [32]:
# Display the 2020 sales DataFrame
print(df_2020.head())

      retailer  retailer_id invoice_date     region         state  \
0  Foot Locker      1185732       1/1/20  Northeast      New York   
1  Foot Locker      1185732       1/1/20  Northeast  Pennsylvania   
2  Foot Locker      1185732       1/1/20  Northeast  Pennsylvania   
3  Foot Locker      1185732       1/1/20  Northeast      New York   
4  Foot Locker      1185732       1/1/20  Northeast  Pennsylvania   

           city                product  price_per_unit  units_sold  \
0      New York  Men's Street Footwear              50        1200   
1  Philadelphia        Women's Apparel              68          83   
2  Philadelphia        Women's Apparel              75         275   
3      New York  Men's Street Footwear              34         384   
4  Philadelphia        Women's Apparel              53          83   

   total_sales  operating_profit sales_method  
0       600000         300000.00     In-store  
1         5644           2426.92       Online  
2       206250      

In [33]:
# Display the 2021 sales DataFrame
print(df_2021.head())

#### Check the data types of each DataFrame

In [53]:
# Check the 2020 sales data types.
print(df_2020.dtypes)

retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object


In [72]:
# Check the 2021 sales data types.
print(df_2021.dtypes)

retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object


#### Combine the sales data by rows.

In [172]:
# Combine the 2020 and 2021 sales DataFrames on the rows and reset the index.
#Using inner join
df_combined = pd.concat([df_2020, df_2021], ignore_index=True)

In [176]:
# Check if any values are null.
null_values = df_combined.isnull().sum()
print(null_values)

retailer            0
retailer_id         0
invoice_date        0
region              0
state               0
city                0
product             0
price_per_unit      0
units_sold          0
total_sales         0
operating_profit    0
sales_method        0
dtype: int64


In [179]:
# Check the data type of each column
print(df_combined.dtypes)

retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object


In [181]:
# Convert the "invoice_date" to a datetime datatype
df_combined['invoice_date'] = pd.to_datetime(df_combined['invoice_date'])

/var/folders/fx/7k4d70096813w2_tf7kjpzgr0000gn/T/ipykernel_38155/418710622.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_combined['invoice_date'] = pd.to_datetime(df_combined['invoice_date'])


In [195]:
# Confirm that the "invoice_date" data type has been changed.
print(df_combined.dtypes)

retailer                    object
retailer_id                  int64
invoice_date        datetime64[ns]
region                      object
state                       object
city                        object
product                     object
price_per_unit               int64
units_sold                   int64
total_sales                  int64
operating_profit           float64
sales_method                object
dtype: object


### 2. Determine which Region Sold the Most Products

In [208]:
# Show the number products sold for region, state, and city.
region_sales = df_combined.groupby(['region', 'state', 'city']).size().reset_index(name='num_products_sold')

# Show the top 5 results.
top_five_regions = region_sales.nlargest(5, 'num_products_sold')

print(top_five_regions)

       region          state          city  num_products_sold
13  Northeast    Connecticut      Hartford                216
17  Northeast  Massachusetts        Boston                216
18  Northeast  New Hampshire    Manchester                216
21  Northeast       New York      New York                216
22  Northeast   Pennsylvania  Philadelphia                216


#### Using `pivot_table`

In [209]:
# Show the number products sold for region, state, and city.
region_sales_pivot = df_combined.pivot_table(index=['region', 'state', 'city'], values='units_sold', aggfunc='sum').reset_index()

# Rename the "units_sold" column to "Total_Products_Sold"
region_sales_pivot = region_sales_pivot.rename(columns={'units_sold': 'Total_Products_Sold'})

# Show the top 5 results.
top_five_regions_pivot = region_sales_pivot.nlargest(5, 'Total_Products_Sold')

print(top_five_regions_pivot)

### 3. Determine which Region had the Most Sales

#### Using `groupby`

In [220]:
# Show the total sales for the products sold for each region, state, and city.
# Groupingby region, state, and city, and summing up the units sold
region_state_city_sales = df_combined.groupby(['region', 'state', 'city'])['units_sold'].sum().reset_index()

# Renaming the "units_sold" column to "Total Sales"
region_state_city_sales = region_state_city_sales.rename(columns={'units_sold': 'Total Sales'})


# Show the top 5 results.
top_five_sales = region_state_city_sales.nlargest(5, 'Total Sales')
print(top_five_sales)

       region       state           city  Total Sales
21  Northeast    New York       New York       111954
33      South       Texas        Houston        90322
44       West  California  San Francisco        85478
43       West  California    Los Angeles        76384
34  Southeast     Florida          Miami        73135


#### Using `pivot_table`

In [231]:
# Show the total sales for the products sold for each region, state, and city.
region_state_city_sales = df_combined.pivot_table(index=['region', 'state', 'city'], values='units_sold', aggfunc='sum')

# Optional: Rename the "total_sales" column to "Total Sales"
region_state_city_sales = region_state_city_sales.rename(columns={'units_sold': 'Total Sales'})

# Show the top 5 results.
top_five_sales = region_state_city_sales.nlargest(5, 'Total Sales')  # Specify the column here!

print(top_five_sales)

                                    Total Sales
region    state      city                      
Northeast New York   New York            111954
South     Texas      Houston              90322
West      California San Francisco        85478
                     Los Angeles          76384
Southeast Florida    Miami                73135


### 4. Determine which Retailer had the Most Sales

#### Using `groupby`

In [240]:
# Show the total sales for the products sold for each retailer, region, state, and city.
# Rename the "total_sales" column to "Total Sales"
retailer_sales = df_combined.groupby('retailer')['units_sold'].sum().reset_index()
retailer_sales = retailer_sales.rename(columns={'units_sold': 'Total Sales'})
# Show the top 5 results.
top_five_retailers = retailer_sales.nlargest(5, 'Total Sales')

print(top_five_retailers)

        retailer  Total Sales
5      West Gear       623840
1    Foot Locker       604369
3  Sports Direct       557640
2         Kohl's       286952
4        Walmart       206225


#### Using `pivot_table`

In [248]:
# Show the total sales for the products sold for each retailer, region, state, and city.
retailer_region_state_city_sales = df_combined.groupby(['retailer', 'region', 'state', 'city'])['units_sold'].sum().reset_index()

# Optional: Rename the "total_sales" column to "Total Sales"
retailer_region_state_city_sales = retailer_region_state_city_sales.rename(columns={'units_sold': 'Total Sales'})

# Show the top 5 results.
top_five_sales = retailer_region_state_city_sales.nlargest(5, 'Total Sales')

print(top_five_sales)

          retailer     region       state           city  Total Sales
103      West Gear       West  California  San Francisco        81233
50          Kohl's       West  California    Los Angeles        74543
22     Foot Locker  Northeast    New York       New York        72196
69   Sports Direct      South       Texas         Dallas        67683
83         Walmart      South       Texas        Houston        65072


### 5. Determine which Retailer Sold the Most Women's Athletic Footwear

In [266]:
# Filter the sales data to get the women's athletic footwear sales data.
womens_athletic_footwear_sales = df_combined[df_combined["product"].str.contains("women|Women|WOMEN|female|Female|FEMALE|lady|Lady|LADY|girl|Girl|GIRL|athletic|Athletic|ATHLETIC|shoe|Shoe|SHOE|footwear|Footwear|FOOTWEAR")]

#### Using `groupby`

In [272]:
# Show the total number of women's athletic footwear sold for each retailer, region, state, and city.
womens_footwear_sales_by_retailer = df_combined.groupby(['retailer', 'region', 'state', 'city'])['units_sold'].sum().reset_index()
# Rename the "units_sold" column to "Womens_Footwear_Units_Sold"
womens_footwear_sales_by_retailer = womens_footwear_sales_by_retailer.rename(columns={'units_sold': 'Womens_Footwear_Units_Sold'})

# Show the top 5 results.
top_5_retailers = womens_footwear_sales_by_retailer.nlargest(5, 'Womens_Footwear_Units_Sold')
print(top_5_retailers)

          retailer     region       state           city  \
103      West Gear       West  California  San Francisco   
50          Kohl's       West  California    Los Angeles   
22     Foot Locker  Northeast    New York       New York   
69   Sports Direct      South       Texas         Dallas   
83         Walmart      South       Texas        Houston   

     Womens_Footwear_Units_Sold  
103                       81233  
50                        74543  
22                        72196  
69                        67683  
83                        65072  


#### Using `pivot_table`

In [277]:
# Show the total number of women's athletic footwear sold for each retailer, region, state, and city.
womens_footwear_sales_pivot = df_combined.pivot_table(index=['retailer', 'region', 'state', 'city'], values='units_sold', aggfunc='sum').reset_index()

# Rename the "units_sold" column to "Womens_Footwear_Units_Sold"
womens_footwear_sales_pivot = womens_footwear_sales_pivot.rename(columns={'units_sold': 'Womens_Footwear_Units_Sold'})

# Show the top 5 results.
top_5_retailers_pivot = womens_footwear_sales_pivot.nlargest(5, 'Womens_Footwear_Units_Sold')
print(top_5_retailers_pivot)

          retailer     region       state           city  \
103      West Gear       West  California  San Francisco   
50          Kohl's       West  California    Los Angeles   
22     Foot Locker  Northeast    New York       New York   
69   Sports Direct      South       Texas         Dallas   
83         Walmart      South       Texas        Houston   

     Womens_Footwear_Units_Sold  
103                       81233  
50                        74543  
22                        72196  
69                        67683  
83                        65072  


### 6. Determine the Day with the Most Women's Athletic Footwear Sales

In [281]:
# Create a pivot table with the 'invoice_date' column is the index, and the "total_sales" as the values.
daily_sales_pivot = df_combined.pivot_table(index='invoice_date', values='units_sold', aggfunc='sum').reset_index()

# Optional: Rename the "total_sales" column to "Total Sales"
daily_sales_pivot = daily_sales_pivot.rename(columns={'units_sold': 'Total Sales'})

# Show the table.
print(daily_sales_pivot)

    invoice_date  Total Sales
0     2020-01-01         2361
1     2020-01-02         2310
2     2020-01-03         2302
3     2020-01-04         2085
4     2020-01-05         1917
..           ...          ...
718   2021-12-27         1314
719   2021-12-28          749
720   2021-12-29          557
721   2021-12-30          442
722   2021-12-31          439

[723 rows x 2 columns]


In [286]:
# Resample the pivot table into daily bins, and get the total sales for each day.
daily_sales_resampled = df_combined.resample('D', on='invoice_date')['units_sold'].sum().reset_index()

# Rename the "units_sold" column to "Total Sales"
daily_sales_resampled = daily_sales_resampled.rename(columns={'units_sold': 'Total Sales'})
# Sort the resampled pivot table in descending order on "Total Sales"
daily_sales_resampled = daily_sales_resampled.sort_values(by='Total Sales', ascending=False)
# Sort the resampled pivot table in descending order on "Total Sales".
print(daily_sales_resampled)

    invoice_date  Total Sales
562   2021-07-16        22857
715   2021-12-16        21770
594   2021-08-17        21285
533   2021-06-17        20768
686   2021-11-17        19126
..           ...          ...
285   2020-10-12            0
286   2020-10-13            0
288   2020-10-15            0
289   2020-10-16            0
365   2020-12-31            0

[731 rows x 2 columns]


### 7.  Determine the Week with the Most Women's Athletic Footwear Sales

In [287]:
# Resample the pivot table into weekly bins, and get the total sales for each week.
weekly_sales_resampled = daily_sales_pivot.resample('W', on='invoice_date')['Total Sales'].sum().reset_index()

# Sort the resampled pivot table in descending order on "Total Sales".
weekly_sales_resampled = weekly_sales_resampled.sort_values(by='Total Sales', ascending=False)
# Show the table
print(weekly_sales_resampled)

   invoice_date  Total Sales
79   2021-07-11        62934
85   2021-08-22        59292
84   2021-08-15        58071
89   2021-09-19        58025
54   2021-01-17        56878
..          ...          ...
46   2020-11-22         2483
48   2020-12-06         2197
50   2020-12-20         1584
49   2020-12-13         1519
41   2020-10-18          896

[105 rows x 2 columns]
